# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [3]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [4]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [5]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [6]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [7]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #1.2!


##### ✅ Answer:
1. Yes, you can change the dimension, by defining the dimensions API parameter, when creating the embedding or after generating it. 
2. OpenAI has achieved this via leveraging Matryoshka Representation Learning (MRL), which encodes information at
different granularities and allows a single embedding to adapt to the computational constraints of downstream tasks.

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [8]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer: Speed and resource optimizations. It can also help parallalize requests instead of sequentially waiting for them to happen. 


So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [9]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer: Yes, you can use message roles, pin model snapshots, build evals to measure prompt performance etc. 


### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [10]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [11]:
print(response)

Hello! The "best" way to write a loop in Python depends on what you're trying to achieve, but I’d be happy to share some general guidelines and examples.

### For simple iterations over sequences (like lists, strings, or ranges), using a `for` loop is usually the most Pythonic and clear:

```python
for item in sequence:
    # do something with item
```

Example:

```python
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(f"I like {fruit}")
```

### If you need to repeat something a fixed number of times, you can use `range()` with a `for` loop:

```python
for i in range(5):
    print(i)
```

### If you need a loop based on a condition (rather than a fixed number of iterations), a `while` loop is appropriate:

```python
count = 0
while count < 5:
    print(count)
    count += 1
```

### Tips for writing good loops:
- Choose `for` loops for iterating over items, `while` loops for conditional repeating.
- Keep the loop body simple and readable.
- Avoid modifying the l

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [12]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [13]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [14]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: The "Michael Eisner Memorial Weak Executive Problem" refers to the situation where a CEO, who previously excelled in a particular function (such as product management, sales, or marketing), hires a weak executive to run that same function. This occurs because the CEO has difficulty letting go of the responsibility that originally brought them success and wants to continue being "the man" in that area. A cited example is Michael Eisner, the CEO of Disney, who had been a brilliant TV network executive. When he bought ABC at Disney, ABC fell to fourth place, and Eisner responded that he could turn it around himself if he had extra time—time he didn't have. This problem highlights how CEOs sometimes deliberately hire weak executives in their former specialty, leading to suboptimal leadership in that function. (Source 1)

Context Count: 3
Similarity Scores: ['Source 1: 0.658', 'Source 2: 0.509', 'Source 3: 0.479']


#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer: The strategy is called prompt engineering
- Define a level of reasoning effort (Controls how much computational effort the model puts into reasoning)
- Define instructions (Developer-level instructions that guide the model's behavior)
- Define text verbosity (Control output verbosity for concise responses)



### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database
- Use a different embedding model
- Add the capability to ingest a YouTube link

While these are suggestions, you should feel free to make whatever augmentations you desire! If you shared an idea during Session 1, think about features you might need to incorporate for your use case! 

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

### Step 1: Implement Multiple Distance Metrics

In [28]:
import numpy as np
from scipy.spatial.distance import euclidean
from typing import List, Tuple

In [35]:
class DistanceMetrics:
    """Collection of three key distance metrics for vector similarity"""
    
    @staticmethod
    def cosine_similarity(vector_a: np.array, vector_b: np.array) -> float:
        """Computes the cosine similarity between two vectors (higher = more similar)"""
        dot_product = np.dot(vector_a, vector_b)
        norm_a = np.linalg.norm(vector_a)
        norm_b = np.linalg.norm(vector_b)
        
        if norm_a == 0 or norm_b == 0:
            return 0.0
        
        return dot_product / (norm_a * norm_b)
    
    @staticmethod
    def euclidean_similarity(vector_a: np.array, vector_b: np.array) -> float:
        """Computes the Euclidean similarity (1 / (1 + distance)) (higher = more similar)"""
        distance = euclidean(vector_a, vector_b)
        return 1 / (1 + distance)
    
    @staticmethod
    def jaccard_similarity(vector_a: np.array, vector_b: np.array) -> float:
        """Computes the Jaccard similarity for binary vectors (higher = more similar)"""
        # Convert to binary vectors (threshold at 0)
        binary_a = (vector_a > 0).astype(int)
        binary_b = (vector_b > 0).astype(int)
        
        intersection = np.sum(binary_a & binary_b)
        union = np.sum(binary_a | binary_b)
        
        if union == 0:
            return 0.0
        
        return intersection / union

# 3 Distance Metrics - Cosine, Euclidean, Jaccard
DISTANCE_METRICS = {
    'cosine_similarity': DistanceMetrics.cosine_similarity,
    'euclidean_similarity': DistanceMetrics.euclidean_similarity,
    'jaccard_similarity': DistanceMetrics.jaccard_similarity
}


### Step 2: Enhanced Vector Database with Multiple Distance Metrics


In [36]:
class EnhancedVectorDatabase(VectorDatabase):
    """Enhanced Vector Database with support for multiple distance metrics"""
    
    def __init__(self, embedding_model=None, default_metric='cosine_similarity'):
        super().__init__(embedding_model)
        self.default_metric = default_metric
        self.available_metrics = DISTANCE_METRICS
    
    def search_with_metric(self, query_vector: np.array, k: int, 
                          distance_metric: str = None) -> List[Tuple[str, float]]:
        """Search using a specific distance metric"""
        metric_name = distance_metric or self.default_metric
        
        if metric_name not in self.available_metrics:
            raise ValueError(f"Unknown metric: {metric_name}. Available: {list(self.available_metrics.keys())}")
        
        metric_func = self.available_metrics[metric_name]
        
        scores = [
            (key, metric_func(query_vector, vector))
            for key, vector in self.vectors.items()
        ]
        
        # Sort based on whether it's a similarity (higher is better) or distance (lower is better)
        if 'distance' in metric_name:
            # For distance metrics, lower is better
            return sorted(scores, key=lambda x: x[1])[:k]
        else:
            # For similarity metrics, higher is better
            return sorted(scores, key=lambda x: x[1], reverse=True)[:k]
    
    def search_by_text_with_metric(self, query_text: str, k: int, 
                                  distance_metric: str = None) -> List[Tuple[str, float]]:
        """Search by text using a specific distance metric"""
        query_vector = self.embedding_model.get_embedding(query_text)
        return self.search_with_metric(query_vector, k, distance_metric)
    
    def compare_metrics(self, query_text: str, k: int = 3) -> dict:
        """Compare results across all available distance metrics"""
        results = {}
        
        for metric_name in self.available_metrics.keys():
            try:
                results[metric_name] = self.search_by_text_with_metric(
                    query_text, k, metric_name
                )
            except Exception as e:
                results[metric_name] = f"Error: {str(e)}"
        
        return results
    
    def get_metric_info(self, metric_name: str) -> dict:
        """Get information about a specific metric"""
        if metric_name not in self.available_metrics:
            return {"error": f"Unknown metric: {metric_name}"}
        
        metric_func = self.available_metrics[metric_name]
        is_distance = 'distance' in metric_name
        
        return {
            "name": metric_name,
            "type": "distance" if is_distance else "similarity",
            "direction": "lower is better" if is_distance else "higher is better",
            "description": metric_func.__doc__ or "No description available"
        }


### Step 3: Enhanced RAG Pipeline with Distance Metric Selection


In [37]:
class EnhancedRAGPipeline(RetrievalAugmentedQAPipeline):
    """Enhanced RAG Pipeline with support for multiple distance metrics"""
    
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: EnhancedVectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False,
                 distance_metric: str = 'cosine_similarity') -> None:
        super().__init__(llm, vector_db_retriever, response_style, include_scores)
        self.distance_metric = distance_metric
    
    def run_pipeline_with_metric(self, user_query: str, k: int = 4, 
                                distance_metric: str = None, **system_kwargs) -> dict:
        """Run the RAG pipeline with a specific distance metric"""
        metric = distance_metric or self.distance_metric
        
        # Retrieve relevant contexts using the specified metric
        context_list = self.vector_db_retriever.search_by_text_with_metric(
            user_query, k=k, distance_metric=metric
        )
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores ({metric}): {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "distance_metric": metric,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }
    
    def compare_metrics_performance(self, user_query: str, k: int = 3) -> dict:
        """Compare performance across different distance metrics"""
        results = {}
        
        for metric_name in self.vector_db_retriever.available_metrics.keys():
            try:
                result = self.run_pipeline_with_metric(
                    user_query, k=k, distance_metric=metric_name
                )
                results[metric_name] = {
                    "response": result["response"],
                    "context_count": result["context_count"],
                    "similarity_scores": result["similarity_scores"],
                    "top_context": result["context"][0][0][:100] + "..." if result["context"] else "No context"
                }
            except Exception as e:
                results[metric_name] = {"error": str(e)}
        
        return results


### Step 4: Demonstration and Testing


In [38]:
# Create enhanced vector database
enhanced_vector_db = EnhancedVectorDatabase()
enhanced_vector_db = asyncio.run(enhanced_vector_db.abuild_from_list(split_documents))

# Create enhanced RAG pipeline
enhanced_rag = EnhancedRAGPipeline(
    vector_db_retriever=enhanced_vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True,
    distance_metric='cosine_similarity'
)

print("Enhanced RAG Pipeline created successfully!")
print(f"Available distance metrics: {list(enhanced_vector_db.available_metrics.keys())}")


Enhanced RAG Pipeline created successfully!
Available distance metrics: ['cosine_similarity', 'euclidean_similarity', 'jaccard_similarity']


In [39]:
# Test different distance metrics on the same query
query = "What is the Michael Eisner Memorial Weak Executive Problem?"

print("=" * 80)
print(f"QUERY: {query}")
print("=" * 80)

# Test the three similarity metrics
test_metrics = ['cosine_similarity', 'euclidean_similarity', 'jaccard_similarity']

for metric in test_metrics:
    print(f"\n--- Using {metric.upper()} ---")
    try:
        result = enhanced_rag.run_pipeline_with_metric(query, k=2, distance_metric=metric)
        print(f"Response: {result['response'][:200]}...")
        print(f"Similarity Scores: {result['similarity_scores']}")
    except Exception as e:
        print(f"Error with {metric}: {e}")


QUERY: What is the Michael Eisner Memorial Weak Executive Problem?

--- Using COSINE_SIMILARITY ---
Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO or startup founder hesitates to fully relinquish control over the function that originally brought them success, ...
Similarity Scores: ['Source 1: 0.654', 'Source 2: 0.504']

--- Using EUCLIDEAN_SIMILARITY ---
Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO or startup founder hires a weak executive in the function that originally brought them success, often because the...
Similarity Scores: ['Source 1: 0.546', 'Source 2: 0.501']

--- Using JACCARD_SIMILARITY ---
Error with jaccard_similarity: '>' not supported between instances of 'list' and 'int'


In [40]:
# Compare all metrics side by side
print("=" * 100)
print("COMPARING ALL DISTANCE METRICS")
print("=" * 100)

comparison_results = enhanced_rag.compare_metrics_performance(query, k=2)

for metric_name, result in comparison_results.items():
    print(f"\n🔍 {metric_name.upper()}")
    print("-" * 50)
    if "error" in result:
        print(f"❌ Error: {result['error']}")
    else:
        print(f"📝 Response: {result['response'][:150]}...")
        print(f"📊 Scores: {result['similarity_scores']}")
        print(f"📄 Top Context: {result['top_context']}")


COMPARING ALL DISTANCE METRICS

🔍 COSINE_SIMILARITY
--------------------------------------------------
📝 Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO or startup founder, who previously excelled in a particular fun...
📊 Scores: ['Source 1: 0.654', 'Source 2: 0.504']
📄 Top Context: ordingly.
Seventh, when hiring the executive to run your former specialty, be
careful you don’t hire...

🔍 EUCLIDEAN_SIMILARITY
--------------------------------------------------
📝 Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO or startup founder hires a weak executive to run the function o...
📊 Scores: ['Source 1: 0.546', 'Source 2: 0.501']
📄 Top Context: ordingly.
Seventh, when hiring the executive to run your former specialty, be
careful you don’t hire...

🔍 JACCARD_SIMILARITY
--------------------------------------------------
❌ Error: '>' not supported between instances of 'list' and 'int'


### Step 5: Metrics Comparison Example



In [41]:
# Quick test: Compare metrics on a sample query
query = "What is the Michael Eisner Memorial Weak Executive Problem?"

print("=" * 60)
print("QUICK METRIC COMPARISON")
print("=" * 60)

for metric in ['cosine_similarity', 'euclidean_similarity', 'jaccard_similarity']:
    try:
        result = enhanced_rag.run_pipeline_with_metric(query, k=2, distance_metric=metric)
        print(f"\n{metric.upper()}:")
        print(f"  Response: {result['response'][:80]}...")
        print(f"  Scores: {result['similarity_scores']}")
    except Exception as e:
        print(f"\n{metric.upper()}: Error - {e}")


QUICK METRIC COMPARISON

COSINE_SIMILARITY:
  Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where...
  Scores: ['Source 1: 0.654', 'Source 2: 0.504']

EUCLIDEAN_SIMILARITY:
  Response: The Michael Eisner Memorial Weak Executive Problem refers to a situation where a...
  Scores: ['Source 1: 0.546', 'Source 2: 0.501']

JACCARD_SIMILARITY: Error - '>' not supported between instances of 'list' and 'int'


In [42]:
# Simplified test with only Cosine and Euclidean metrics
query = "What is the Michael Eisner Memorial Weak Executive Problem?"

print("=" * 60)
print("SIMPLIFIED METRIC COMPARISON")
print("=" * 60)

for metric in ['cosine_similarity', 'euclidean_similarity']:
    try:
        result = enhanced_rag.run_pipeline_with_metric(query, k=2, distance_metric=metric)
        print(f"\n{metric.upper()}:")
        print(f"  Response: {result['response'][:80]}...")
        print(f"  Scores: {result['similarity_scores']}")
    except Exception as e:
        print(f"\n{metric.upper()}: Error - {e}")


SIMPLIFIED METRIC COMPARISON

COSINE_SIMILARITY:
  Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where...
  Scores: ['Source 1: 0.654', 'Source 2: 0.504']

EUCLIDEAN_SIMILARITY:
  Response: The "Michael Eisner Memorial Weak Executive Problem" refers to the situation whe...
  Scores: ['Source 1: 0.546', 'Source 2: 0.501']


In [43]:
# Example usage:
result = enhanced_rag.run_pipeline_with_metric("What is the best managerial strategy for a startup?", distance_metric='cosine_similarity')


In [44]:
print(result)

{'response': 'The best managerial strategy for a startup, based on the provided context, centers on aggressively seeking product/market fit within a big market rather than detailed upfront planning, hiring executives only when truly necessary, and focusing on execution amid inevitable uncertainties and adjustments.\n\nKey points include:\n\n1. Prioritize Achieving Product/Market Fit:\n- Successful startups almost always reach product/market fit, despite potentially "screwing up all kinds of other things" such as marketing plans, sales models, or HR policies along the way (Source 3).\n- Startups should aggressively pursue a big market and product/market fit after launch, rather than trying to perfectly plan in advance, because the initial idea and plan are highly uncertain and likely to evolve rapidly (Source 4).\n\n2. Careful and Timely Hiring of Executives:\n- Startups often hire executives too early, especially before product/market fit is achieved; this can result in expensive hires